In [ ]:
import pandas as pd
import numpy as np
import unicodedata

# Primeiro, vamos pegar todos os dados e organizá-los da melhor forma possível
lista = pd.read_html('https://internationaloffice.usp.br/consultaconvenios?Pais=&Tipo=&Interveni%EAncia=&Institui%E7%E3o+Exterior=&orderby=Pais&btnAction=Buscar')

# Mas os dados foram organizados de uma forma estranha.
# Vamos então organizar um pouco melhor
x  = np.reshape(lista, (3*len(lista[0]))).T
data = pd.Series(x)

# Remover os NaN e resetar os índices
data = data.dropna().reset_index(drop=True)

# Resultados iniciais
data.head(20)

0                                                 País:
1                                         África do Sul
2                                                 Tipo:
3                                    Convênio Acadêmico
4                                       Interveniência:
5                                                    FE
6                                 Instituição Exterior:
7     SCHOOL OF EDUCATION - UNIVERSITY OF KWAZULU-NATAL
8                                               Objeto:
9     Cooperação acadêmica; intercâmbio de docentes/...
10                                  Áreas Contempladas:
11                                            Educação.
12                                            Vigência:
13                         De 23/10/2020 até 22/10/2025
14                                                País:
15                                        África do Sul
16                                                Tipo:
17                                   Convênio Ac

In [ ]:
"""
    De fato a lista não ajuda a analisar os dados.

    Vamos então transformar isso numa tabela tradicional e salvar os dados.
"""

# Encontrar índices de linhas que contêm os titulos
titulos = [i for i, item in enumerate(data) if item.endswith(":")]

# Criar uma lista de dicionários para armazenar os pares de titulos e dados
lista_dados = []
dados_atuais = {}

for i in range(len(titulos)):
    start = titulos[i]
    titulo = data[start][:-1]
    if i + 1 < len(titulos):
        end = titulos[i+1]
        valores = data[start+1:end]
    else:
        valores = data[start+1:]

    if titulo in dados_atuais:
        lista_dados.append(dados_atuais)
        dados_atuais = {}

    dados_atuais[titulo] = " ".join(valores)

# Criar o DataFrame a partir da lista de dicionários
df = pd.DataFrame(lista_dados)

# Simplificar os dados de vigência
df["Vigência"] = df["Vigência"].str.slice(18)

# Limpeza dos nomes, tirando caracteres especiais
df.columns = [unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode() for val in df.columns.str.lower()]
for coluna in df.columns:
    df[coluna] = df[coluna].apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())

# Salvando o resultado...
df.to_csv('DadosAucani.csv', index=False, sep=';')

# Tabela final
df.head(16)

,pais,tipo,interveniencia,instituicao exterior,objeto,areas contempladas,vigencia
0,Africa do Sul,Convenio Academico,FE,SCHOOL OF EDUCATION - UNIVERSITY OF KWAZULU-NATAL,Cooperacao academica; intercambio de docentes/...,Educacao.,22/10/2025
1,Africa do Sul,Convenio Academico,IRI,DEPARTMENT OF INTERNATIONAL RELATIONS - UNIVER...,Cooperacao academica; intercambio de docentes/...,Relacoes Internacionais,14/08/2025
2,Africa do Sul,Acordo de Cooperacao,ESALQ,MANGOSUTHU UNIVERSITY OF TECHNOLOGY,Cooperacao academica.,"Ciencias Agricolas, Ciencias da Gestao e Engen...",22/04/2024
3,Africa do Sul,Acordo de Cooperacao,USP,UNIVERSITY OF CAPE TOWN,Cooperacao academica.,Areas de mutuo interesse.,13/12/2026
4,Africa do Sul,Acordo de Cooperacao,USP,UNIVERSITY OF ZULULAND,Cooperacao Academica.,Areas de mutuo interesse.,03/07/2028
5,Africa do Sul,Convenio Academico,IRI,FACULTY OF ARTS & SOCIAL SCIENCES - STELLENBOS...,Cooperacao academica; intercambio de docentes/...,Relacoes Internacionais,24/02/2024
6,Alemanha,Acordo de Cooperacao,ICMC,CARL VON OSSIETZKY UNIVERSITAT OLDENBURG,Cooperacao academica.,Areas do ICMC-USP e do Institute of Mathematic...,28/07/2025
7,Alemanha,Convenio Academico,EESC,UNIVERSITAT KASSEL,Cooperacao academica; intercambio de docentes/...,Engenharia,15/05/2024
8,Alemanha,Convenio Academico,IME,TECHNISCHE UNIVERSITAT MUNCHEN,Cooperacao academica; intercambio de estudante...,Areas de interesse do IME-USP e da TUM.,25/07/2024
9,Alemanha,Convenio Academico,FFCLRP,TECHNISCHE UNIVERSITAT CAROLO-WILHELMINA,O presente convenio tem por objeto a cooperaca...,Ciecnia da Computacao,15/09/2025
